#ΑΝΑΚΤΗΣΗ ΠΛΗΡΟΦΟΡΙΑΣ PROJECT
##ΟΝΟΜΑΤΕΠΩΝΥΜΑ:1.ΚΟΥΣΑΘΑΝΑΣ ΑΝΤΩΝΙΟΣ, 2.ΖΓΟΥΛΕΤΑΣ ΣΤΕΡΓΙΑΝΟΣ
##ΑΜ1:713242017058 , ΑΜ2:19390060
##LINK GITHUB : https://github.com/antoniskous/AnaktisiProject

#CRAWLER2.PY AΡΧΕΙΟ

In [ ]:
#ΦΟΡΤΩΣΗ ΒΙΒΛΙΟΘΗΚΩΝ ΚΑΙ ΠΡΟΕΠΕΞΕΡΓΑΣΙΑ ΚΕΙΜΕΝΟΥ
import requests
from bs4 import BeautifulSoup
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import nltk

# Εγκατάσταση nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
##ΠΡΟΕΠΕΞΕΡΓΑΣΙΑ ΚΕΙΜΕΝΟΥ
def preprocess_text(text):
    """
    Επεξεργάζεται το κείμενο:
    - Αφαιρεί μη-αλφαριθμητικούς χαρακτήρες.
    - Χωρίζει το κείμενο σε tokens.
    - Αφαιρεί stopwords.
    - Εφαρμόζει lemmatization.
    """
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text.lower())

    # Stop-word removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

# Παράδειγμα
example_text = "Artificial Intelligence is the future of technology!"
print("Preprocessed tokens:", preprocess_text(example_text))


#ΑΝΑΖΗΤΗΣΗ ΚΑΙ ΕΞΑΓΩΓΗ ΑΡΘΡΩΝ. ΠΙΟ ΣΥΓΚΕΚΡΙΜΕΝΑ ΤΟ 'crawl_wikipedia' είναι για την αναζήτηση και το 'extract_wikipedia_article' για την εξαγωγή των τίτλων.

In [ ]:
def crawl_wikipedia(query, max_results=5):
   
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json",
        "srlimit": max_results
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        search_results = response.json().get('query', {}).get('search', [])
        articles = []
        
        for result in search_results:
            title = result['title']
            page_url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
            articles.append(extract_wikipedia_article(page_url))
        
        return articles
    else:
        print(f"Failed to fetch search results for query: {query}")
        return []

def extract_wikipedia_article(url):
   
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        title = soup.find('h1', {'id': 'firstHeading'}).text.strip()
        content = ''
        for paragraph in soup.find_all('p'):
            content += paragraph.text.strip() + ' '

        preprocessed_title = preprocess_text(title)
        preprocessed_content = preprocess_text(content)

        print(f"Title: {title}, Preprocessed Title: {preprocessed_title}")
        print(f"Content Sample: {content[:100]}, Preprocessed Content: {preprocessed_content[:10]}")

        article = {
            'url': url,
            'title': title,
            'content': content,
            'preprocessed_title': preprocessed_title,
            'preprocessed_content': preprocessed_content
        }

        return article
    else:
        print(f"Failed to fetch article from {url}")
        return None


#ΔΗΜΙΟΥΡΓΙΑ ΚΑΙ ΑΠΟΘΗΚΕΥΣΗ ΑΝΤΡΙΣΤΡΟΦΟΥ ΕΥΡΕΤΗΡΙΟΥ 'create_and_save_reverse_index'

In [ ]:
def create_and_save_reverse_index(articles):
  
    reverse_index = defaultdict(list)

    for idx, article in enumerate(articles):
        for term in set(article['preprocessed_title'] + article['preprocessed_content']):
            reverse_index[term].append(idx)

    print(f"Reverse Index Sample: {dict(list(reverse_index.items())[:10])}")
    save_reverse_index(reverse_index)

def save_to_json(articles, filename='wikipedia_articles.json'):
   
    with open(filename, 'w') as json_file:
        json.dump(articles, json_file, indent=2)
    print(f'Saved {len(articles)} articles to {filename}')

def save_reverse_index(reverse_index, filename='reverse_index.json'):
  
    with open(filename, 'w') as json_file:
        json.dump(reverse_index, json_file, indent=2)
    print(f'Saved reverse index to {filename}')


#ΕΚΤΕΛΕΣΗ ΠΑΡΑΔΕΙΓΜΑΤΟΣ

In [ ]:
# Εκτέλεση του crawler
query = "Artificial Intelligence"
wikipedia_articles = crawl_wikipedia(query)

# Αποθήκευση άρθρων και ευρετηρίου
save_to_json(wikipedia_articles)
create_and_save_reverse_index(wikipedia_articles)


#ΠΡΟΕΠΕΞΕΡΓΑΣΙΑ ΚΕΙΜΕΝΟΥ
ΒΑΣΙΚΕΣ ΛΕΙΤΟΥΡΓΙΕΣ:
1.ΑΦΑΙΡΕΣΗ ΜΗ-ΑΛΦΑΡΙΘΜΗΤΙΚΩΝ ΧΑΡ/ΡΩΝ
2.TOKENIZATION
3.AΦΑΙΡΕΣΗ STOPWORDS.
4.LEMMATIZATION

In [ ]:
#SEARCH.PY AΡΧΕΙΟ
#ΕΙΣΑΓΩΓΗ ΒΙΒΛΙΟΘΗΚΩΝ ΚΑΙ ΦΟΡΤΩΣΗ ΔΕΔΟΜΕΝΩΝ
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from math import log
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
import nltk

# Εγκατάσταση nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Φόρτωση δεδομένων
def load_articles(filename='wikipedia_articles.json'):
    with open(filename, 'r') as json_file:
        articles = json.load(json_file)
    return articles

def load_reverse_index(filename='reverse_index.json'):
    with open(filename, 'r') as json_file:
        reverse_index = json.load(json_file)
    return reverse_index

articles = load_articles()
reverse_index = load_reverse_index()

print(f"Loaded {len(articles)} articles.")
print(f"Sample reverse index keys: {list(reverse_index.keys())[:5]}")

In [7]:
def preprocess_text(text):
  
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens


##ΥΛΟΠΟΙΗΣΗ ΑΛΓΟΡΙΘΜΩΝ ΑΝΑΖΗΤΗΣΗΣ
1.BOOLEAN
2.VSM
3.OKAPI BM25


In [4]:
def boolean_query_processing(query, reverse_index):
   
    query = query.upper()
    query_tokens = re.findall(r'[\w]+|AND|OR|NOT|\(|\)', query)
    term_postings = {}
    for token in query_tokens:
        if token not in {'AND', 'OR', 'NOT', '(', ')'}:
            preprocessed_token = preprocess_text(token)[0]
            if preprocessed_token in reverse_index:
                term_postings[token] = set(reverse_index[preprocessed_token])
            else:
                term_postings[token] = set()

    def eval_query(tokens):
        stack = []
        for token in tokens:
            if token == "AND":
                b = stack.pop()
                a = stack.pop()
                stack.append(a & b)
            elif token == "OR":
                b = stack.pop()
                a = stack.pop()
                stack.append(a | b)
            elif token == "NOT":
                a = stack.pop()
                stack.append(set(range(len(reverse_index))) - a)
            else:
                stack.append(term_postings.get(token, set()))
        return stack[0]

    def infix_to_postfix(tokens):
        precedence = {'NOT': 3, 'AND': 2, 'OR': 1}
        output = []
        operators = []
        for token in tokens:
            if token not in precedence and token not in {'(', ')'}:
                output.append(token)
            elif token == '(':
                operators.append(token)
            elif token == ')':
                while operators and operators[-1] != '(':
                    output.append(operators.pop())
                operators.pop()
            else:
                while (operators and operators[-1] != '(' and
                       precedence.get(token, 0) <= precedence.get(operators[-1], 0)):
                    output.append(operators.pop())
                operators.append(token)
        while operators:
            output.append(operators.pop())
        return output

    postfix_query = infix_to_postfix(query_tokens)
    return list(eval_query(postfix_query))


##ΠΑΡΑΔΕΙΓΜΑ ΕΚΤΕΛΕΣΗΣ

In [ ]:
# Boolean Retrieval
query = "artificial AND intelligence OR learning"
result_docs = boolean_query_processing(query, reverse_index)
print(f"Boolean Retrieval αποτελέσματα: {result_docs}")

# Vector Space Model
query = "artificial intelligence"
vsm_results = vector_space_model(query, reverse_index, articles)
print(f"VSM αποτελέσματα: {vsm_results}")

# Okapi BM25
bm25_results = okapi_bm25(query, reverse_index, articles)
print(f"BM25 αποτελέσματα: {bm25_results}")


#ΣΥΜΠΕΡΑΣΜΑΤΑ
Η ΜΕΘΟΔΟΣ BOOLEAN EINAI ΧΡΗΣΙΜΗ ΓΙΑ ΑΚΡΙΒΕΙΣ ΕΡΩΤΗΣΕΙΣ.
ΤΟ OKAPI BM 25 ΕΙΝΑΙ ΧΡΗΣΙΜΟ ΓΙΑ ΕΡΩΤΗΜΑΤΑ ΠΟΛΛΑΠΛΩΝ ΛΕΞΕΩΝ
ΤΟ VSM XΡΗΣΙΜΟΠΟΙΕΙΤΑΙ ΚΥΡΙΩΣ ΓΙΑ ΜΕΓΑΛΑ ΕΡΩΤΗΜΑΤΑ